In [1]:
import serial, time, csv, math, pandas as pd, pyvisa, glob, matplotlib.pyplot as plt, os.path, numpy as np, pathlib, datetime;
from csv import reader
import sys

In [2]:
#pip install pandas pip install pyvisa -U pyvisa pyusb pyserial -U pyvisa-py

In [3]:
#establish connection thru pyvisa, find and connect to available visa connections
rm = pyvisa.ResourceManager()
print(rm.list_resources())

('USB0::0x2A8D::0x0201::MY57701346::INSTR', 'USB0::0xF4EC::0xEE38::SDG2XCAC1R0004::INSTR')


In [4]:
#set up connections

#assign visa object siglent function generator to fg
fg = rm.open_resource('USB0::0xF4EC::0xEE38::SDG2XCAC1R0004::INSTR')
print (fg.query('*IDN?'))

#assign visa object keysight digital multimeter to dmm
dmm = rm.open_resource('USB0::0x2A8D::0x0201::MY57701346::INSTR')
print (dmm.query('*IDN?'))

Siglent Technologies,SDG2122X,SDG2XCAC1R0004,2.01.01.22R5

Keysight Technologies,34470A,MY57701346,A.02.17-02.40-02.17-00.52-04-01



In [5]:
#data scan 

#freq scanning range
dx=1

#set up clock frequency on function generator channel 2
fclk = 0 #10 khz fclk = 50 hz center freq
fg.write("C2:BSWV AMP,5")
fg.write("C2:BSWV WVTP,SQUARE")
fg.write("C2:OUTP ON")

#set up channel 1 on function generator 
fvin = 0 
fg.write("C1:BSWV AMP,5") #Vin is set to 5 Vpp and does not change
fg.write("C1:OUTP ON")

bpname = ""

while bpname != "quit":
    
    #ask user for bpf serial #
    bpname = input("Enter bandpass filter serial # in format ### or enter 'quit':")
    ct = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
    fg.write("C1:OUTP ON")
    fg.write("C2:OUTP ON")
    
    #make a folder for each new bpf
    if bpname == 'quit':
        sys.exit()
    
    new_dir = pathlib.Path("/Volumes/GoogleDrive/.shortcut-targets-by-id/1OcwVYdCFsNitauuq7YD82J3wd781RFEz/Zhao_Nicole /jupyter/BPF_results/test", 
                           'SN_'+ bpname + '_' + str(ct)) 
    new_dir.mkdir(parents=True)

    for c in range (10000,70000,20000): #set up loop for fclk
        frange=[]
        fclk = c
        fc = fclk/200 #center freq 
        sweep = int(0.3 * fc) #sweep freq +/- 0.3*fc 
        fg.write("C2:BSWV FRQ," + str(fclk))
        fvin_min = int(fc - sweep) #fvin set to start at fc-sweep
        fvin_max = int(fvin_min + 2*sweep) #fvin set to end at fc+sweep 
        fg.write("C1:BSWV FRQ," + str(fvin_min)) #starting freq; will be set to sweep later
        #print(fg.query("*OPC?"))
        
        #set up csv file inside folder
        #fname = str("bp_" + str(bpname) + "_" + str(c//200) + "hz.csv") '{:0>2}'.format(fc)
        fname = str("bp_" + str(bpname) + "_" + str(ct) + "_" + str('{:0>3}'.format(c//200)) + "hz.csv") 
        fpath = (new_dir / fname ).with_suffix('.csv')
        with fpath.open(mode='w+') as csvfile:
            logfile = csv.writer(csvfile)
            logfile.writerow(["BP freq (Hz)","Vin freq (Hz)", "Vout freq dmm (Hz)", "Vin (V)", "Vout dmm (V)"])

            #combine a fine & coarse list to scan more points near the center 
            fine = [fc+dx*x for x in [-3,-2,-1,-0.5,0,0.5,1,2,3]]
            coarse=np.linspace(fvin_min,fvin_max,30,dtype=int)
            new=np.concatenate([coarse,fine])
            [frange.append(x) for x in new if x not in frange] 
            frange.sort()
            #print (frange)
            
            for x in frange:
            
            #for x in range (fvin_min, fvin_max, 1): #set up loop for freq sweep
                fg.write("C1:BSWV FRQ," + str(x))
                readback_c1 = fg.query("C1:BSWV?")
                readback_c2 = fg.query("C2:BSWV?")
                #split readback into a list
                data_list_c1 = readback_c1.split(",")
                data_list_c2 = readback_c2.split(",")
                bp_freq = int(data_list_c2[3][:5])//200
                
                #pause for 1 second
                #time.sleep(1)
                #read measurements from digital multimeter
                vout_dmm = dmm.query_ascii_values("MEAS:VOLT:AC?")[0]*math.sqrt(2)*2 
                fout_dmm = dmm.query_ascii_values("MEAS:FREQ?")  

                #write to csv file
                logfile.writerow([str(bp_freq), str(x) , str(fout_dmm[0]), data_list_c1[7][0], str(vout_dmm)])

    if int(fg.query("*OPC?")) == 1:
        fg.write("C1:OUTP OFF")
        fg.write("C2:OUTP OFF")
        print("sweep completed for bpf#" + str(bpname))
    
#remember to turn off power before switchin filter
#3.33 - 3.36

Enter bandpass filter serial # in format ### or enter 'quit':114
sweep completed for bpf#114
Enter bandpass filter serial # in format ### or enter 'quit':118
sweep completed for bpf#118
Enter bandpass filter serial # in format ### or enter 'quit':119
sweep completed for bpf#119
Enter bandpass filter serial # in format ### or enter 'quit':120
sweep completed for bpf#120
Enter bandpass filter serial # in format ### or enter 'quit':121
sweep completed for bpf#121
Enter bandpass filter serial # in format ### or enter 'quit':121
sweep completed for bpf#121
Enter bandpass filter serial # in format ### or enter 'quit':122
sweep completed for bpf#122
Enter bandpass filter serial # in format ### or enter 'quit':123
sweep completed for bpf#123
Enter bandpass filter serial # in format ### or enter 'quit':124
sweep completed for bpf#124
Enter bandpass filter serial # in format ### or enter 'quit':125
sweep completed for bpf#125
Enter bandpass filter serial # in format ### or enter 'quit':126
sweep

SystemExit: 

/Users/porpoise/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#plotting code

#find intersection point with horizontal 3dB line
def find_roots(x,y): 
    s = np.abs(np.diff(np.sign(y))).astype(bool)
    return x[:-1][s] + np.diff(x)[s]/(np.abs(y[1:][s]/y[:-1][s])+1) 

homedir = '/Volumes/GoogleDrive/.shortcut-targets-by-id/1OcwVYdCFsNitauuq7YD82J3wd781RFEz/Zhao_Nicole /jupyter/BPF_results/test/012422/'
os.chdir(homedir)
#make a list of all folders
dirs = [d for d in os.listdir(homedir) if os.path.isdir(os.path.join(homedir, d))]

for d in dirs: 
    os.chdir(homedir + '/' + d) 
    #define plot space
    fig, ax = plt.subplots(1,1, figsize=(12,8))
    print (d)
    #create list to store x & y values pulled from files
    xs = [] #bandpass freq
    ys = [] #vout/vin
    zs = [] #store Vin freq (Hz) values
    colors = ['g','y','c']
    output = [] #output value for text file
    
    #write values into text file
    with open ("data_" + d, 'w') as file: 
        file.write("# SN, Timestamp, f0Nominal_Hz, f0Meas_Hz, BW_3dB, Ratio\n")
        file.close

    #import and read file
    file_list = glob.glob(homedir + '/' + d + '/*.csv')
    file_list.sort() 
    for file in file_list:
        data = pd.read_csv(file)
        print(file)
        xs.append(data["BP freq (Hz)"][0])
        ys.append(data["Vout dmm (V)"]/data["Vin (V)"])
        zs.append(data["Vin freq (Hz)"]) 

    #iterate over each center freq
    for ifc in range(len(xs)): 
        xmin = min(zs[ifc]) #establish min & max freq range on xaxis based on data set
        xmax = max(zs[ifc]) 

        #have values interpolated
        f = interpolate.interp1d(zs[ifc],ys[ifc],'cubic')
        x_interp = np.arange(xmin,xmax,0.01) #map to 1/100 hz 
        y_interp = f(x_interp) #map y against x
        
        #plot real data as points, plot interp data as ---
        plt.scatter(zs[ifc],ys[ifc],s=8,c=colors[ifc])
        #plot interp data as line
        plt.plot(x_interp, y_interp, color=colors[ifc], linestyle='dashed',label= str(xs[ifc]) + 'hz',alpha=0.5)
        
        #plot zero
        roots = find_roots(x_interp,10**(-3/20)-y_interp) #find roots at y=-3dB
        ax.axhline(y=10**(-3/20),color='red',ls=":") #plot 3dB line in red
        ax.plot(roots,np.interp(roots, x_interp, y_interp),color="b") #plot 3dB bandwidth in blue 

        for root in roots:
            ax.axvline(x=root,color="red",ls=":") #graph vertical line at each zero

        #find important values 
        print ("np.max(y):"+ str(np.max(y_interp)))
        f0 = x_interp[np.argmax(y)] #xmax; 1 decimal place 
        print(f0)
        BW3dB = float(np.diff(roots)) #find 3dB bandwidth
        ratio = f0/BW3dB #ratio of f0/BW3dB
        plt.text(400,0.004+0.02*ifc,"f_0=" + "{:.2f}".format(f0) + "Hz"  + "\nBW_3dB=" 
               + "{:.2f}".format(BW3dB) + "Hz" + "\nRatio=" + "{:.2f}".format(ratio),fontsize=10, bbox = dict(facecolor = 'blue', alpha = 0.2,edgecolor='blue')) 

        #write data into file for each bpf
        output.append([d[3:6],d[7:],xs[ifc],"{:.2f}".format(f0),"{:.2f}".format(BW3dB), 
                       "{:.2f}".format(ratio)])
        print(output)
    #store output value in text file
    
    with open('data_' + d, 'a') as f:
        #f.writelines(output)
        #f.close()
        writer = csv.writer(f)
        writer.writerows(output)

    # Set tick font size
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(16)
    
    ax.set_title("bode_" + d, fontsize=16)
    ax.set_xlabel("Frequency (Hz)", fontsize=16)
    ax.set_ylabel("$V_{out}/V_{in}$", fontsize=16)
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlim(10,1000)
    ax.set_ylim(10**-2.5,1.3)
    plt.legend(prop={"size":16})
    plt.savefig("bode_" + d + ".png")
    plt.show()
    plt.clf()